In [1]:
import RDatasets

In [2]:
iris = RDatasets.dataset("datasets", "iris"); # a DataFrame

In [3]:
using MLJ

In [4]:
y, X = unpack(iris, ==(:Species), colname -> true);

In [5]:
first(X, 3) |> pretty

┌─────────────┬────────────┬─────────────┬────────────┐
│ SepalLength │ SepalWidth │ PetalLength │ PetalWidth │
│ Float64     │ Float64    │ Float64     │ Float64    │
│ Continuous  │ Continuous │ Continuous  │ Continuous │
├─────────────┼────────────┼─────────────┼────────────┤
│ 5.1         │ 3.5        │ 1.4         │ 0.2        │
│ 4.9         │ 3.0        │ 1.4         │ 0.2        │
│ 4.7         │ 3.2        │ 1.3         │ 0.2        │
└─────────────┴────────────┴─────────────┴────────────┘


In [6]:
models(matching(X,y))

47-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :docstring, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :input_scitype, :target_scitype, :output_scitype), T} where T<:Tuple}:
 (name = AdaBoostClassifier, package_name = ScikitLearn, ... )
 (name = AdaBoostStumpClassifier, package_name = DecisionTree, ... )
 (name = BaggingClassifier, package_name = ScikitLearn, ... )
 (name = BayesianLDA, package_name = MultivariateStats, ... )
 (name = BayesianLDA, package_name = ScikitLearn, ... )
 (name = BayesianQDA, package_name = ScikitLearn, ... )
 (name = BayesianSubspaceLDA, package_name = MultivariateStats, ... )
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_name 

In [7]:
Tree = @load DecisionTreeClassifier pkg=DecisionTree

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/sandhya/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


import MLJDecisionTreeInterface ✔


MLJDecisionTreeInterface.DecisionTreeClassifier

In [18]:
typeof(Tree)

DataType

In [8]:
tree = Tree()

DecisionTreeClassifier(
    max_depth = -1,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = 0,
    post_prune = false,
    merge_purity_threshold = 1.0,
    pdf_smoothing = 0.0,
    display_depth = 5) @509

In [9]:
evaluate(tree, X, y,
                resampling=CV(shuffle=true), measure=log_loss, verbosity=0)

┌───────────────────────────────────┬───────────────┬───────────────────────────
│ _.measure                         │ _.measurement │ _.per_fold               ⋯
├───────────────────────────────────┼───────────────┼───────────────────────────
│ \eLogLoss{Float64} @633\e │ 2.88          │ [2.22e-16, 4.33, 4.33, 2 ⋯
└───────────────────────────────────┴───────────────┴───────────────────────────
                                                                1 column omitted
_.per_observation = [[[2.22e-16, 2.22e-16, ..., 2.22e-16], [36.0, 2.22e-16, ..., 2.22e-16], [2.22e-16, 2.22e-16, ..., 2.22e-16], [2.22e-16, 2.22e-16, ..., 2.22e-16], [2.22e-16, 2.22e-16, ..., 2.22e-16], [2.22e-16, 2.22e-16, ..., 2.22e-16]]]
_.fitted_params_per_fold = [ … ]
_.report_per_fold = [ … ]


In [10]:
evaluate(tree, X, y,
                resampling=CV(shuffle=true), measure=accuracy, operation=predict_mode, verbosity=0)

┌───────────────┬───────────────┬────────────────────────────────────┐
│ _.measure     │ _.measurement │ _.per_fold                         │
├───────────────┼───────────────┼────────────────────────────────────┤
│ Accuracy @560 │ 0.92          │ [0.92, 0.96, 0.92, 1.0, 0.92, 0.8] │
└───────────────┴───────────────┴────────────────────────────────────┘
_.per_observation = [missing]
_.fitted_params_per_fold = [ … ]
_.report_per_fold = [ … ]


In [11]:
typeof(tree)

MLJDecisionTreeInterface.DecisionTreeClassifier

In [12]:
mach = machine(tree, X, y)

Machine{DecisionTreeClassifier,…} @691 trained 0 times; caches data
  args: 
    1:	Source @935 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @239 ⏎ `AbstractVector{Multiclass{3}}`


In [13]:
train, test = partition(eachindex(y), 0.7, shuffle=true); # 70:30 split

In [14]:
fit!(mach, rows=train);

┌ Info: Training Machine{DecisionTreeClassifier,…} @691.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342


In [15]:
yhat = predict(mach, X[test,:]);

In [16]:
yhat[3:5]

3-element MLJBase.UnivariateFiniteVector{Multiclass{3}, String, UInt8, Float64}:
 UnivariateFinite{Multiclass{3}}(setosa=>1.0, versicolor=>0.0, virginica=>0.0)
 UnivariateFinite{Multiclass{3}}(setosa=>1.0, versicolor=>0.0, virginica=>0.0)
 UnivariateFinite{Multiclass{3}}(setosa=>1.0, versicolor=>0.0, virginica=>0.0)

In [17]:
log_loss(yhat, y[test]) |> mean

1.6019401506274291